# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data_group = data.groupby(["CustomerID","ProductID","ProductName"]).agg({'Quantity':"sum"})
data_group

Quantity
CustomerID ProductID ProductName                              
33         23        Crab - Imitation Flakes                 1
           51        Wine - Crozes Hermitage E.              1
           53        Cassis                                  1
           67        Wine - Sogrape Mateus Rose              1
           70        Squid - Tubes / Tenticles 10/20         1
...                                                        ...
98200      392       Puree - Passion Fruit                  25
           408       Blueberries                            25
           418       Truffle Cups - Brown                   25
           422       Garlic - Primerba, Paste               25
           444       Cheese - Cambozola                     25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(data, values='Quantity',index=["ProductID","ProductName"],columns=['CustomerID'], aggfunc=np.sum,fill_value=0)
table

,CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductID,ProductName,,,,,,,,,,,,,,,,,,,,,
1,Flour - Whole Wheat,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,25,0,0,25
2,Cookie Chocolate Chip With,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Onions - Cippolini,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"Sauce - Gravy, Au Jus, Mix",0,1,0,0,0,0,1,1,0,1,...,25,0,0,0,0,0,0,0,0,0
5,Artichokes - Jerusalem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,Gatorade - Xfactor Berry,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
449,Nantuket Peach Orange,0,0,0,0,0,0,0,0,0,0,...,0,0,25,0,25,0,25,0,0,0
450,Wine - Vidal Icewine Magnotta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,25,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)

display(distances)


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similarities = distances.loc[33].sort_values(ascending=False)[1:6]
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
similarities.index

Int64Index([264, 3535, 3317, 2503, 3305], dtype='int64', name='CustomerID')

In [9]:
top5 = data_group.loc[similarities.index]
top5

Quantity
CustomerID ProductID ProductName                           
264        2         Cookie Chocolate Chip With           1
           30        Cheese - Brie, Triple Creme          1
           48        Pecan Raisin - Tarts                 1
           49        Bacardi Breezer - Tropical           1
           59        Pastry - Butterscotch Baked          1
...                                                     ...
3535       423       Fish - Scallops, Cold Smoked         1
           429       Beef - Short Loin                    1
           438       Juice - Orange                       1
           439       Beef - Chuck, Boneless               1
           444       Cheese - Cambozola                   1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
total_products = top5.groupby('ProductName').agg({'Quantity':"sum"}).sort_values(by="Quantity", ascending=False)
total_products

total_products

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
#total_products = pd.DataFrame(total_products).reset_index().loc


In [12]:
customer_33 = table[33]
customer_33

ProductID  ProductName                    
1          Flour - Whole Wheat                0
2          Cookie Chocolate Chip With         0
3          Onions - Cippolini                 0
4          Sauce - Gravy, Au Jus, Mix         0
5          Artichokes - Jerusalem             0
                                             ..
448        Gatorade - Xfactor Berry           0
449        Nantuket Peach Orange              0
450        Wine - Vidal Icewine Magnotta      0
451        Soup - Campbells Tomato Ravioli    0
452        Napkin White - Starched            0
Name: 33, Length: 452, dtype: int64

In [13]:
#recomendados = [i for i in data6.index if i not in data7.ProductName]

selected = []

for i in total_products.index:
    if i not in customer_33:
        selected.append(i)

selected = selected[0:5]
selected


['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Towels - Paper / Kraft',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [14]:
#Creamos un lista con todos los cutomer IDs y un diccionario con valores a 0
# Filtramos los 20 primeros valores para facilitar el cálculo

customer_ids = list(data["CustomerID"][0:20])
recommendation = { i: 0 for i in customer_ids}

In [15]:
# Definimos una función para aplicar
def recommendations(lista=customer_ids):
    
    for i in lista:
        similarities = distances.loc[i].sort_values(ascending=False)[1:6]
        top5 = data_group.loc[similarities.index]
        total_products = top5.groupby('ProductName').agg({'Quantity':"sum"}).sort_values(by="Quantity", ascending=False)
        customer_x = table[i]
        
        selected = []
        
        for e in total_products.index:
            if e not in customer_x:
                selected.append(e)
        
    for i in lista:
        recommendation[i]  = selected[0:5]
        
    return recommendation


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [16]:
cust_reco = recommendations()
custo_df = pd.DataFrame(cust_reco)
custo_df

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,86595,46715,91013,69931,19753,9546,10224,72846,28976
0,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium,Beef - Ground Medium
1,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey,Tea - Earl Grey
2,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries,Blueberries
3,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger,Beef - Texas Style Burger
4,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan,Rambutan


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [17]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cityblock'))), 
                         index=table.columns, columns=table.columns)